In [1]:
import pandas as pd
from citipy import citipy
import requests
import json
import numpy as np
import gmaps
from pprint import pprint
from config import gkey
gmaps.configure(api_key=gkey)

In [39]:
citi_data = pd.read_csv('../WeatherPy/citi_data.csv')
fig = gmaps.figure(layout = {'width':'800px', 'height':'400px'}, zoom_level=1, center =(0,0))
heat = gmaps.heatmap_layer(citi_data[['latitude','longitude']], weights=citi_data['humidity'], max_intensity=100, point_radius=5)
fig.add_layer(heat)
fig


Figure(layout=FigureLayout(height='400px', width='800px'))

In [99]:
ideal_temp = (citi_data['temperature']>70) & (citi_data['temperature']<80)
ideal_windspeed = citi_data['windspeed']<10
ideal_cloudiness = citi_data['cloudiness']<25
vacation_spots = citi_data[ideal_temp & ideal_windspeed & ideal_cloudiness].copy(deep=True)
vacation_spots
# a =vacation_spots.iloc[0,:]
# a[['latitude','longitude']].astype(float)



,Unnamed: 0,name,latitude,longitude,temperature,cloudiness,humidity,windspeed
91,100,jiuquan,39.7432,98.5174,75.34,2.0,13.0,2.26
102,112,yumen,40.2833,97.2000,77.56,2.0,13.0,6.93
120,132,sabha,27.0377,14.4283,79.97,12.0,17.0,2.89
125,137,gat,31.6100,34.7642,74.39,0.0,52.0,0.74
163,180,baghdad,33.3406,44.4009,75.11,0.0,35.0,3.29
173,192,sinjar,36.3209,41.8754,74.28,0.0,35.0,1.39
184,208,baghdad,33.3406,44.4009,75.11,0.0,35.0,3.29
248,280,sakakah,29.9697,40.2064,72.97,0.0,20.0,4.83
253,288,waddan,29.1614,16.1390,75.40,1.0,29.0,3.78
271,308,puerto ayora,-0.7393,-90.3518,74.55,23.0,91.0,7.00


In [108]:
hotels = pd.DataFrame(columns=['name', 'city', 'country', 'latitude','longitude'])
hotels['city'] = vacation_spots['name']
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
parameters ={
    'key':gkey,
    'location':'-0.7393, -90.3518',
    'keyword':'hotel',
    'radius':5000,
}

for index,place in vacation_spots.iterrows():
    parameters['location']=f'{place["latitude"]},{place["longitude"]}'
    response = requests.get(base_url,params=parameters).json()
    try:
        hotels.loc[index,'name']=response['results'][0]['name']
        hotels.loc[index, 'latitude'] = response['results'][0]['geometry']['location']['lat']
        hotels.loc[index, 'longitude'] = response['results'][0]['geometry']['location']['lng']
    except IndexError as e:
        print(e)
hotels


list index out of range
list index out of range
list index out of range
list index out of range


,name,city,country,latitude,longitude
91,Jiuquan Hotel,jiuquan,NaN,39.7374,98.5078
102,NaN,yumen,NaN,NaN,NaN
120,فندق علي,sabha,NaN,27.0496,14.4119
125,Desert Gat,gat,NaN,31.6027,34.7643
163,Baghdad Hotel,baghdad,NaN,33.3187,44.4151
173,NaN,sinjar,NaN,NaN,NaN
184,Baghdad Hotel,baghdad,NaN,33.3187,44.4151
248,Raoum Inn Hotel,sakakah,NaN,29.9453,40.1812
253,NaN,waddan,NaN,NaN,NaN
271,Galapagos Habitat by Eco Luxury Group,puerto ayora,NaN,-0.742633,-90.3098


In [111]:
hotels.dropna(subset=['name'], inplace=True)
# hotels

In [112]:
markers = gmaps.marker_layer(hotels[['latitude','longitude']])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='400px', width='800px'))